# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [554]:
# load json as string
a=json.load((open('data/world_bank_projects_less.json')))

In [555]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

*****
## Exercise 1. 10 countries with most project

In [556]:
df=pd.read_json('data/world_bank_projects.json')
solution1=df.countrycode.value_counts().head(10)
solution1

ID    19
CN    19
VN    17
IN    16
RY    13
BD    12
NP    12
MA    12
3A    11
MZ    11
Name: countrycode, dtype: int64

## Exercise 2. 10 Major Project Themes

In [557]:
with open('data/world_bank_projects.json') as data_file:    
    data = json.load(data_file)
df2 = json_normalize(data,'mjtheme_namecode')
#Finding number of project themes by code (some names are in Blank)
df3=df2.groupby(by=['code']).count()
#Finding the pairs {code: name} 
df2=df2.sort_values(by='name')
values=df2.groupby(by=['code']).last()
#Merging to show the Thems name with the total number of appeareance of the corresponding code
solution2=values.merge(df3, how='outer', left_on='code', right_on='code', suffixes =['_Theme', '_Count']).sort_values(by='name_Count', ascending=False)
solution2=solution2.head(10)
solution2


,name_Theme,name_Count
code,,
11,Environment and natural resources management,250
10,Rural development,216
8,Human development,210
2,Public sector governance,199
6,Social protection and risk management,168
4,Financial and private sector development,146
7,Social dev/gender/inclusion,130
5,Trade and integration,77
9,Urban development,50


****
## Exercise 3. DataFrame with missing names Filled-in

#### Solution3_1

In [558]:
#Mergin df2 with dictionary Values{code: name} from previous exercise
df4=values.merge(df2, how='outer', left_on='code', right_on='code')
#Taking only the 'name' column merged from values
solution3_1=df4.iloc[slice(None),:2]
solution3_1.columns=['code', 'name']
solution3_1.groupby(by=['code', 'name']).code.count()


code  name                                        
1     Economic management                              38
10    Rural development                               216
11    Environment and natural resources management    250
2     Public sector governance                        199
3     Rule of law                                      15
4     Financial and private sector development        146
5     Trade and integration                            77
6     Social protection and risk management           168
7     Social dev/gender/inclusion                     130
8     Human development                               210
9     Urban development                                50
Name: code, dtype: int64

#### Solution3_2

In [593]:
a=df2
a.sort_values(by=['code','name'],ascending=True, inplace=True)
a.name.where(a.name!='', inplace=True)
a.name.fillna(method='ffill',inplace=True)
a.groupby(by=['code','name'])['code'].count()

code  name                                        
1     Economic management                              38
10    Rural development                               216
11    Environment and natural resources management    250
2     Public sector governance                        199
3     Rule of law                                      15
4     Financial and private sector development        146
5     Trade and integration                            77
6     Social protection and risk management           168
7     Social dev/gender/inclusion                     130
8     Human development                               210
9     Urban development                                50
Name: code, dtype: int64